# Clinical Evidence Researcher Agent with Strands
In this notebook we create the clinical evidence researcher agent using the open-source Strands agents framework

#### Install Strands agents and required dependencies

In [ ]:
%pip install strands-agents strands-agents-tools xmltodict --quiet

#### Ensure the latest version of boto3 is shown below
Ensure the boto3 version printed below is **1.37.1** or higher.

In [ ]:
%pip show boto3

#### Import required libraries

In [ ]:
import os
import boto3
import json
import uuid
import requests
from typing import Dict, Any, List
from strands import Agent, tool
from strands.models import BedrockModel
from strands_tools import retrieve

from utils.PubMed import PubMed

# Initialize KB tool variable
kb_tool = None

# Get AWS account information
sts_client = boto3.client('sts')
account_id = sts_client.get_caller_identity()['Account']
region = boto3.Session().region_name

## Prerequisites

Run through the notebook environment setup in [00-setup_environment.ipynb](00-setup_environment.ipynb).

#### Setup AWS clients
Define the clients to AWS services that will be used by tools.

In [ ]:
# Initialize AWS clients
bedrock_client = boto3.client('bedrock-runtime', region_name=region)
bedrock_agent_client = boto3.client("bedrock-agent", region_name=region)

print(f"Region: {region}")
print(f"Account ID: {account_id}")

#### Setup Knowledge Base for internal evidence retrieval
In this example we are going to use a built-in tool from Strands agents called **`retrieve`**. This tool semantically retrieve data from Amazon Bedrock Knowledge Bases for RAG, memory, and other purposes. The tool requires the knowledge base id, which we'll be provided throught the environment variable `KNOWLEDGE_BASE_ID` defined below.

In [ ]:
# Find the Knowledge Base
response = bedrock_agent_client.list_knowledge_bases()

# Iterate through knowledge bases and find needed one
ncbi_kb_id = None
for kb in response['knowledgeBaseSummaries']:
    kb_name = kb['name']
    if 'ncbiKnowledgebase' in kb_name:
        ncbi_kb_id = kb['knowledgeBaseId']
        break

if ncbi_kb_id:
    print(f"Found Knowledge Base ID: {ncbi_kb_id}")
    os.environ["KNOWLEDGE_BASE_ID"] = ncbi_kb_id
    print("Knowledge Base will be integrated using direct Strands tool approach")
else:
    print("Warning: Knowledge Base not found. Internal evidence retrieval may not work.")
    ncbi_kb_id = None

# Strands Agent Creation
In this section we create the agent using the Strands framework

#### Define agent configuration and instructions

In [ ]:
clinical_research_agent_name = "Clinical-evidence-researcher-strands"
clinical_research_agent_description = "Research internal and external evidence using Strands framework"
clinical_research_agent_instruction = """You are a medical research assistant AI specialized in summarizing internal and external 
evidence related to cancer biomarkers. Your primary task is to interpret user queries, gather internal and external 
evidence, and provide relevant medical insights based on the results. Use only the appropriate tools as required by 
the specific question. Always use the retrieve knowledge base tool first for internal evidence search. Follow these instructions carefully: 

1. Use the retrieve tool to search internal evidence. Use the query PubMed tool after you performed a search using the retrieve tool.

2. When querying PubMed: 
   a. Summarize the findings of each relevant study with citations to the specific pubmed web link of the study 
   b. The json output will include 'Link', 'Title', 'Summary'. 
   c. Always return the Title and Link (for example, 'https://pubmed.ncbi.nlm.nih.gov/') of each study in your response.  

3. For internal evidence, make use of the knowledge base to retrieve relevant information. 
   Always provide citations to specific content chunks. 

4. When providing your response: 
   a. Start with a brief summary of your understanding of the user's query. 
   b. Explain the steps you're taking to address the query. Ask for clarifications from the user if required. 
   c. Separate the responses generated from internal evidence (knowledge base) and external evidence (PubMed api).  
   d. Conclude with a concise summary of the findings and their potential implications for medical research.
"""

#### Define tools for Strands agent
We are going to use a custom tool to query PubMed combined with the retrieve tool from Strands framework. Retrieve tool doesn't need to be defined by a function like custom tools, so we simply add it to the list of tools.

In [ ]:
# Define the tools using Strands @tool decorator
@tool
def query_pubmed(query: str) -> str:
    """
    Query PubMed for relevant biomedical literature based on the user's query.
    This tool searches PubMed abstracts and returns relevant studies with titles, links, and summaries.
    
    Args:
        query (str): The search query for PubMed
    
    Returns:
        str: JSON string containing PubMed search results with titles, links, and summaries
    """
    
    pubmed = PubMed()

    print(f"\nPubMed Query: {query}\n")
    result = pubmed.run(query)
    print(f"\nPubMed Results: {result}\n")
    return result

# Create list of custom tools
clinical_research_agent_tools = [query_pubmed, retrieve]
print(f"Created {len(clinical_research_agent_tools)} custom tools for the Strands agent")

#### Setup AWS Bedrock provider for Strands

In [ ]:
# Create Bedrock model for Strands
model = BedrockModel(
    model_id="anthropic.claude-3-5-sonnet-20241022-v2:0",
    region_name=region,
    temperature=0.1,
    streaming=False
)

#### Create the Strands agent

In [ ]:
# Create the Strands agent
try:
    # Use the custom tools we created
    clinical_evidence_agent = Agent(
        model=model,
        tools=clinical_research_agent_tools,
        system_prompt=clinical_research_agent_instruction
    )
    
    print(f"Successfully created Strands agent: {clinical_research_agent_name}")
    print(f"Agent has {len(clinical_research_agent_tools)} tools available:")
    for tool in clinical_research_agent_tools:
        print(f"  - {tool.__name__}")
    
except Exception as e:
    print(f"Error creating agent: {e}")
    raise

#### Test the Strands agent

In [ ]:
# Test the agent with a research query
test_query = "Can you search PubMed for evidence around the effects of biomarker use in oncology on clinical trial failure risk"

print(f"Testing agent with query: {test_query}")
print("=" * 140)

try:
    # Run the agent
    response = clinical_evidence_agent(test_query)
    
except Exception as e:
    print(f"Error during agent execution: {e}")
    import traceback
    traceback.print_exc()

#### Advanced usage examples

In [ ]:
# Example of more complex queries
complex_queries = [
    "Search for evidence on LRIG1 biomarker in lung cancer prognosis",
    "Find studies about biomarker-guided therapy in precision oncology",
    "What does the internal knowledge base say about molecular phenotypes and imaging?"
]

def test_complex_query(query: str):
    """
    Test a complex query with the agent
    """
    print(f"\nTesting query: {query}")
    print("-" * 100)
    
    try:
        response = clinical_evidence_agent(query)
    except Exception as e:
        print(f"Error: {e}")

for query in complex_queries: 
    test_complex_query(query)

#### Session management and conversation continuity

In [ ]:
# Demonstrate conversation continuity
def interactive_research_session():
    """
    Simple interactive session with the clinical evidence researcher agent
    """
    print("Interactive Clinical Evidence Research Session")
    print("Ask about biomarkers, clinical trials, or cancer research")
    print("Type 'quit' to exit")
    print("=" * 60)
    
    while True:
        user_input = input("\nYour research question: ")
        
        if user_input.lower() in ['quit', 'exit', 'q']:
            print("Research session ended.")
            break
            
        try:
            response = clinical_evidence_agent(user_input)                
        except Exception as e:
            print(f"Error: {e}")

interactive_research_session()

## Summary

This notebook demonstrated how to build an agent with Strands framework that connects to a Bedrock Knowledge and to the PubMed APIs.

### Tools Available:
- `query_pubmed`: Searches PubMed for biomedical literature
- `KnowledgeBase`: Retrieves internal evidence from the NCBI knowledge base

### Research Capabilities:
- **Medical literature search** via PubMed API integration
- **Internal evidence retrieval** from curated knowledge base
- **Comprehensive research synthesis** combining multiple sources
- **Citation tracking** with links to original studies
- **Biomarker research** specialization for cancer research
